In [62]:
import os
import pandas as pd
import numpy as np
import json
from plotting_utils import read_data_from_logs, clean_results, MAX_BUDGETS
from performance_profile import get_workloads_time_to_target, BASE_WORKLOADS, get_base_workload_name

In [63]:
# Open json file to read heldout workloads
with open("held_out_workloads_algoperf_v05.json", "r") as f:
  HELDOUT_WORKLOADS = json.load(f)

SELF_TUNING = True
log_path = "../../submissions_algorithms_v0.5/logs/algoperf_scoring_v05/"
if SELF_TUNING:
  log_path = os.path.join(log_path, "self_tuning")
else:
  log_path = os.path.join(log_path, "external_tuning")

full_results = read_data_from_logs(log_path)
clean_results_df = clean_results(full_results)

Reading data for submission: prize_qualification_baseline
Reading data for submission: schedule_free_adamw
Reading data for submission: AdamG
Reading data for submission: sinv6_75
Reading data for submission: sinv6
Reading data for submission: nadamw_sequential


In [64]:
def get_workload_runtimes(full_results):
  dfs = []
  for submission_name, submission_results in full_results.items():
    # Get time to targets for each submission across studies and trials
    dfs.append(
        get_workloads_time_to_target(
            submission_results,
            submission_name,
            "score",
            self_tuning_ruleset=SELF_TUNING))
  df = pd.concat(dfs).transpose()
  # Split into base and held-out workloads
  df_base = df.loc[BASE_WORKLOADS]
  df_heldout = df.loc[HELDOUT_WORKLOADS]
  return df_base, df_heldout


runtimes_base, runtimes_heldout = get_workload_runtimes(full_results)

In [65]:
def normalize_by_budget(row):
  workload_name = row.name
  if workload_name not in MAX_BUDGETS:
    workload_name = get_base_workload_name(workload_name)
  budget = MAX_BUDGETS[workload_name]
  return row / budget


def convert_df_to_strings_with_rounding(df):
  """Converts all numeric values in a DataFrame to strings rounded to two significant digits.

    Args:
        df: The DataFrame to modify.

    Returns:
        The modified DataFrame with all numeric values converted to strings.
    """
  df = df.copy()

  # Get the indices of numeric columns
  numeric_cols = df.select_dtypes(include='number').columns

  # Convert numeric values to strings with rounding
  for col in numeric_cols:
    df[col] = df[col].apply(lambda x: f"{x:.2f}")
    df[col] = df[col].astype(str)  # Explicitly convert to string

  return df

def latex_style_base_inf(value):
  if str(value) != "inf" and str(value) != "nan" and not str(value).startswith(r"\textcolor"):
    return r"\textcolor{TUdark_light}{" + f"{str(value)}" + r"\textsuperscript{\textdaggerdbl}}"
  else:
    return str(value)

def latex_style_heldout_inf(value):
  if str(value) != "inf" and str(value) != "nan" and not str(value).startswith(r"\textcolor"):
    return r"\textcolor{TUdark_light}{" + f"{str(value)}" + r"\textsuperscript{\textdagger}}"
  else:
    return str(value)

def latex_style_slow(value):
  # Check if it isn't already ignored
  if not str(value).startswith(r"\textcolor"):
    return r"\textcolor{TUdark_light}{" + f"{str(value)}" + r"\textsuperscript{\textasteriskcentered}}"
  else:
    return value

def disqualify_scores(base, heldout, base_latex, heldout_latex):
  # 1. Base -> Heldout
  # Identify cells in base that are inf or nan
  base_mask = base.isin([np.inf, -np.inf]) | base.isna()
  # For cells where base is inf or nan, ignore the corresponding cell in heldout
  heldout_latex[base_mask] = heldout_latex[base_mask].applymap(latex_style_heldout_inf)
  heldout[base_mask] = np.inf

  # 2. Check for <4x fastest
  # Identify cells that are the fastest in each row
  base_min = base.min(axis=1)
  heldout_min = heldout.min(axis=1)
  # Identify cells that are too slow but check that they are not inf
  too_slow_mask_base = base.apply(lambda x: x > 4 * base_min, axis=0) & ~base.isin([np.inf, -np.inf])
  too_slow_mask_heldout = heldout.apply(lambda x: x > 4 * heldout_min, axis=0) & ~heldout.isin([np.inf, -np.inf])
  base_latex[too_slow_mask_base] = base_latex[too_slow_mask_base].applymap(latex_style_slow)
  heldout_latex[too_slow_mask_heldout] = heldout_latex[too_slow_mask_heldout].applymap(latex_style_slow)
  base[too_slow_mask_base] = np.inf
  heldout[too_slow_mask_heldout] = np.inf

  # 3. Heldout -> Base
  # Identify cells in heldout that are inf or nan
  heldout_mask = heldout.isin([np.inf, -np.inf]) | heldout.isna()
  # For cells where heldout is inf or nan, ignore the corresponding cell in base
  base_latex[heldout_mask] = base_latex[heldout_mask].applymap(latex_style_base_inf)
  base[heldout_mask] = np.inf
 
  return base_latex, heldout_latex


def process_runtimes(base, heldout):
  base = base.apply(normalize_by_budget, axis=1).sort_values("workload")
  heldout = heldout.apply(normalize_by_budget, axis=1).sort_values("workload")

  # Rename heldout workloads to base workloads
  heldout = heldout.rename(index=get_base_workload_name)

  # Create dataframe copies (for latex output)
  base_latex = convert_df_to_strings_with_rounding(base)
  heldout_latex = convert_df_to_strings_with_rounding(heldout)

  # Highlight fastest submission per workload
  smallest_index_base = base.idxmin(axis=1)
  smallest_index_heldout = heldout.idxmin(axis=1)

  # Prefix the smallest value with "\B"
  for index, col in smallest_index_base.items():
    base_latex.loc[index, col] = f"\B {base_latex.loc[index, col]}"
  for index, col in smallest_index_heldout.items():
    heldout_latex.loc[index, col] = f"\B {heldout_latex.loc[index, col]}"

  base_latex, heldout_latex = disqualify_scores(base, heldout, base_latex, heldout_latex)

  # Transpose dataframes
  base_latex = base_latex.transpose()
  heldout_latex = heldout_latex.transpose()

  # Mark workloads with "H.O." or "Base"
  heldout_latex.columns = pd.MultiIndex.from_product(
      [heldout_latex.columns, ['H.O.']])
  base_latex.columns = pd.MultiIndex.from_product([base_latex.columns, ['Base']])

  # Merge
  latex_df = pd.concat([base_latex, heldout_latex], axis=1).sort_index(
      axis=1, level=0)
  # Sort rows alphabetically
  latex_df.rename(index={"prize_qualification_baseline": "baseline"}, inplace=True)
  latex_df = latex_df.sort_index(axis=0)

  # Cleanup
  latex_df.index.name = None
  latex_df.columns.names = [None, None]
  workload_rename_dict = {
      "criteo1tb": r"\criteo",
      "fastmri": r"\fastmri",
      "imagenet_resnet": r"\resnet",
      "imagenet_vit": r"\vit",
      "librispeech_conformer": r"\conformer",
      "librispeech_deepspeech": r"\makecell{\textsc{Deep}\\ \textsc{Speech}}",
      "ogbg": r"\ogbg",
      "wmt": r"\wmt",
  }
  submission_rename_dict = {
      "shampoo_submission": r"\shampoo \newline \textsc{Submission}",
      "baseline": r"\baseline",
      "caspr_adaptive": r"\textsc{CASPR} \newline \textsc{Adaptive}",
      "schedule_free_adamw": r"\sfadam",
      "schedule_free_prodigy": r"\sfprodigy",
      "amos": r"\amos",
      "lawa_ema": r"\lawaema",
      "lawa_queue": r"\lawaq",
      "cyclic_lr": r"\cycliclr",
      "generalized_adam": r"\generalizedadam",
      "nadamp": r"\nadamp",
      "AdamG": r"\adamg",
      "sinv6_75": r"\sinvnum",
      "sinv6": r"\sinv",
      "nadamw_sequential": r"\textsc{NadamW}\newline \textsc{Sequential}",
  }
  latex_df.rename(columns=workload_rename_dict, inplace=True)
  latex_df.rename(index=submission_rename_dict, inplace=True)
  display(latex_df)

  return latex_df


latex_df = process_runtimes(runtimes_base, runtimes_heldout)

/tmp/ipykernel_43646/1781783613.py:54: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  heldout_latex[base_mask] = heldout_latex[base_mask].applymap(latex_style_heldout_inf)
/tmp/ipykernel_43646/1781783613.py:64: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  base_latex[too_slow_mask_base] = base_latex[too_slow_mask_base].applymap(latex_style_slow)
/tmp/ipykernel_43646/1781783613.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  heldout_latex[too_slow_mask_heldout] = heldout_latex[too_slow_mask_heldout].applymap(latex_style_slow)
/tmp/ipykernel_43646/1781783613.py:73: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  base_latex[heldout_mask] = base_latex[heldout_mask].applymap(latex_style_base_inf)


\criteo  \
                                                                                          Base   
\adamg                                                                                     inf   
\baseline                                                                                 0.75   
\textsc{NadamW}\newline \textsc{Sequential}  \textcolor{TUdark_light}{2.96\textsuperscript{...   
\sfadam                                                                                \B 0.75   
\sinv                                                                                      nan   
\sinvnum                                                                                   nan   

                                                                                                \
                                                                                          H.O.   
\adamg                                                                                     inf   
\baseline                                                                              \B 0.07   
\textsc{NadamW}\newline \textsc{Sequential}  \textcolor{TUdark_light}{0.57\textsuperscript{...   
\sfadam                                                                                   0.25   
\sinv                                                                                      nan   
\sinvnum                                                                                   nan   

                                            \fastmri           \
                                                Base     H.O.   
\adamg                                           inf      inf   
\baseline                                       0.22     0.51   
\textsc{NadamW}\newline \textsc{Sequential}     0.27  \B 0.44   
\sfadam                                      \B 0.15     0.58   
\sinv                                           0.49     0.87   
\sinvnum                                        0.45     0.80   

                                                                                       \resnet  \
                                                                                          Base   
\adamg                                                                                     inf   
\baseline                                    \textcolor{TUdark_light}{\B inf\textsuperscrip...   
\textsc{NadamW}\newline \textsc{Sequential}                                                inf   
\sfadam                                                                                    inf   
\sinv                                                                                      inf   
\sinvnum                                                                                   inf   

                                                                                                \
                                                                                          H.O.   
\adamg                                                                                     inf   
\baseline                                    \textcolor{TUdark_light}{\B inf\textsuperscrip...   
\textsc{NadamW}\newline \textsc{Sequential}                                                inf   
\sfadam                                                                                    inf   
\sinv                                                                                      inf   
\sinvnum                                                                                   inf   

                                                \vit \conformer  \
                                                Base       Base   
\adamg                                           inf        inf   
\baseline                                       0.95    \B 0.94   
\textsc{NadamW}\newline \textsc{Sequential}     1.58        inf   
\sfadam                                      \B 0.68       0.97   
\sinv                   

In [66]:
def process_latex_code(latex_df):
  filename = 'plots/runtimes_'
  if SELF_TUNING:
    filename += 'self_tuning.tex'
  else:
    filename += 'external_tuning.tex'
  print(f"Writing to {filename}")
  with open(filename, 'w') as tf:
    latex_code = latex_df.to_latex(escape=False)

    # Modify latex code
    latex_code = latex_code.replace(r"\begin{tabular}{lllllllllllllll}", r"{\renewcommand{\arraystretch}{1.25}\setlength{\tabcolsep}{4pt}\begin{tabularx}{0.95\textwidth}{Xlglglgllgllglg}")
    latex_code = latex_code.replace(r"\end{tabular}", r"\end{tabularx}}")
    latex_code = latex_code.replace(r"\B inf", r"inf")  # ignore "fastest run" for all infs
    latex_code = latex_code.replace(r" & Base & H.O. & Base & H.O. & Base & H.O. & Base & Base & H.O. & Base & Base & H.O. & Base & H.O. \\", r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-8} \cmidrule(lr){9-10} \cmidrule(lr){11-11} \cmidrule(lr){12-13} \cmidrule(lr){14-15} & Base & H.O. & Base & H.O. & Base & H.O. & Base & Base & H.O. & Base & Base & H.O. & Base & H.O. \\")
    latex_code = latex_code.replace(r"\multicolumn{2}{r}{", r"\multicolumn{2}{c}{")
    latex_code = latex_code.replace("nan", r"\textcolor{TUdark_light}{\texttt{NaN}}")
    latex_code = latex_code.replace("inf", r"\textcolor{TUdark_light}{\texttt{inf}}")
    

    # Write to file
    tf.write(latex_code)

process_latex_code(latex_df)

Writing to plots/runtimes_self_tuning.tex
